In [40]:
import numpy as np
import qiskit as qk

#### Second Quantized Hamiltonian
$H = \sum_{ij}  \kappa_{ij} a_i^\dagger a_j + \sum_{ijkl} \nu_{ijkl} a_i^\dagger a_j^\dagger a_k a_l$

In [41]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, Molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.7414]]],
                     charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')


from qiskit.chemistry.transformations import (FermionicTransformation, 
                                              FermionicTransformationType, 
                                              FermionicQubitMappingType)

# Define Transformation
fermionic_transformation = FermionicTransformation(
            transformation=FermionicTransformationType.FULL,
            qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER,
            two_qubit_reduction=False,
            freeze_core=False)

qubit_op, _ = fermionic_transformation.transform(driver) # transform molecule info
print(qubit_op)

SummedOp([
  -0.8126179630230753 * IIII,
  0.17119774903432938 * IIIZ,
  -0.22278593040418504 * IIZI,
  0.17119774903432944 * IZII,
  -0.22278593040418504 * ZIII,
  0.12054482205301814 * IIZZ,
  0.16862219158920955 * IZIZ,
  0.045322202052874044 * XXYY,
  0.045322202052874044 * YYYY,
  0.045322202052874044 * XXXX,
  0.045322202052874044 * YYXX,
  0.16586702410589216 * ZIIZ,
  0.16586702410589216 * IZZI,
  0.17434844185575685 * ZIZI,
  0.12054482205301814 * ZZII
])


In [42]:
# Jordan Wigner Transformation to get H operator
N = 4 # number of qubits, excluding ancilla
N_f = 2 # number of Fermions
n_max = 2 # max excitations

# Evaluate the number of computational-basis states(particle-hole)
n_basis_states = 0
for n in range(n_max):
    n_basis_states += math.comb(N-N_f, n+1) * math.comb(N_f,n+1)
    

# initialize computational basis
comp_basis = np.zeros((n_basis_states,N),dtype = int) 

# construct reference states
for i in range(N_f):
    for j in range(n_basis_states):
        comp_basis[j][i] = 1

In [43]:
def S_comb(m,n):
    a = np.zeros(n-m+1,dtype=int)
    for i in range(n-m+1):
        a[i] = m+i
    return a


def D_comb(m,n): # for D calculation
    a = np.zeros((int((n-m)*(n-m+1)/2),2),dtype=int)
    x = 0
    for i in range(m,n):
        for j in range(i+1,n+1):
            a[x][0] = i
            a[x][1] = j
            x +=1
    return a

# returns all combinations of sites to change from reference states
low_S = S_comb(1,N_f)
high_S = S_comb(N_f+1,N)

low_D = D_comb(1,N_f)
high_D = D_comb(N_f+1,N)

# construct full computational basis
ind = 0
for low in range(low_S.size):     # S : only 1 fermion gets excited at a time
    for high in range(high_S.size):
        comp_basis[ind][low_S[low]-1] = 0    # particle moves from
        comp_basis[ind][high_S[high]-1] = 1  # low to high
        print(comp_basis[ind][:])
        ind += 1

for low in range(low_D.shape[0]): # D : 2 fermions get excited at a time
    for high in range(high_D.shape[0]):
        comp_basis[ind][low_D[low][0]-1] = 0 
        comp_basis[ind][low_D[low][1]-1] = 0
        comp_basis[ind][high_D[high][0]-1] = 1
        comp_basis[ind][high_D[high][1]-1] = 1
        print(comp_basis[ind][:])
        ind += 1

[0 1 1 0]
[0 1 0 1]
[1 0 1 0]
[1 0 0 1]
[0 0 1 1]


In [44]:
backend = qk.Aer.get_backend('qasm_simulator')
n_shots = 2**12

# Evaluate diagonal elements of the effective Hamiltonian
def diag_elements(basis_vec, pauli_op, n_shots):
    num_qb = pauli_op.num_qubits
    # initialise circuit
    qc = qk.QuantumCircuit(num_qb,num_qb)
    
    # apply n-gate (encode basis state through X-gates)
    for i in range(num_qb):
        if (basis_vec[i]): 
            qc.x(i)

    # apply pauli operators to encoded circuit
    qc = qc + pauli_op.to_circuit()
    qc.measure(range(num_qb),range(num_qb))
    
    exp_values = qk.execute(qc, backend, shots=n_shots)
    results = exp_values.result().get_counts()
    return results


def pauli_eigval(pauli_op):
    # return array of eigenvalues of Pauli string in binary order
    pauli_str = pauli_op.primitive
    a = np.linalg.eigh(pauli_str.to_matrix()) #eigvals
    eig_val = np.zeros(len(a[0]))
    for i in range(len(a[0])):
        eig_val[np.where(a[1][:,i] == 1)] = a[0][i]
    return eig_val
    
    
# called at the end of circuit function
def exp_value(eig, results, n_shots):
    avg = 0.
    # for every result that was measured
    for a in results.keys():
        # obtain index by converting binary measurement to integer
        b = 0
        for i in range(len(a)):
             b += int(a[len(a)-1-i]) * 2**i
        # weighted sum of eigenvalues and number of measurments
        avg += eig[b]*results[a]
    return avg / n_shots

For the diagonal elements of H

$<n|H|n> = \sum_i \lambda_i <n|h_i|n>$

In [45]:
# Diagonal Elements
H_diag = np.zeros(len(fermionic_transformation.untapered_qubit_op.basis),dtype=float)

# iterate over Pauli strings
for i_pauli in range(len(fermionic_transformation.untapered_qubit_op.basis)):
    pauli_eig = pauli_eigval(qubit_op[i_pauli]) # get eigenvalues of pauli string
    
    # iterate over every basis vector
    for i_basis in range(comp_basis[:][0].size):
        # evaluate circuit for every basis vector and pauli string
        results = diag_elements(comp_basis[i_basis][:], qubit_op[i_pauli], n_shots)
        for i_results in results.keys():  # take exp value of results and multiply by JW coefficients
            H_diag[int(i_results,2)] += qubit_op[i_pauli].coeff * exp_value(pauli_eig, results, n_shots)

print(H_diag)

[ 0.          0.          0.          0.          0.         -1.83043838
 -1.06494419  0.          0.         -1.06494419 -0.25450366  0.
  0.          0.          0.        ]
